In [1]:
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import numpy as np

In [2]:
df = pd.read_parquet('data/CaseRigshospitalet_optimized.parquet')
df.head()

,Patient ID,Patientkontakt ID,Patient alder på kontaktstart tidspunkt,Kontakt startdato,Kontakt slutdato,Kontakttype,Indlæggelsesmåde,Patientkontakttype,Aktionsdiagnosekode,Bidiagnosekode,...,Procedure udført,Behandlingskontakt ID,Besøgstype,Patient køn,Patient civilstand,Patient oprettet på Min SP (J/N),Patient land,Patient region,Patient postnummer,Patient kommune
0,Z4115614,1301023075465,0,2023-01-13 12:32:00,2023-01-13 13:02:00,Fysisk fremmøde,Planlagt,2 Ambulant,DZ001,,...,2023-01-13 12:32:00,176.794.528,OBST EFTERFØDSEL 30,Mand,Ugift,J,Danmark,Hovedstaden (84),2100,København
1,Z4115614,1301023121831,0,2023-01-15 13:27:00,2023-01-15 13:57:00,Fysisk fremmøde,Planlagt,2 Ambulant,DZ001,,...,2023-01-15 13:27:00,177.035.032,OBST EFTERFØDSEL 30,Mand,Ugift,J,Danmark,Hovedstaden (84),2100,København
2,Z4119225,1301023357249,0,2023-01-30 10:20:00,2023-01-30 10:50:00,Fysisk fremmøde,Planlagt,2 Ambulant,DZ001,,...,2023-01-30 10:20:00,178.269.449,OBST EFTERFØDSEL 30,Mand,Ugift,J,Danmark,Hovedstaden (84),2100,København
3,Z4127033,1301023443807,0,2023-02-04 12:46:00,2023-02-04 13:16:00,Fysisk fremmøde,Planlagt,2 Ambulant,DZ001,,...,2023-02-04 12:46:00,178.703.583,OBST EFTERFØDSEL 30,Mand,Ugift,J,Danmark,Hovedstaden (84),2100,København
4,Z4127033,1301023443875,0,2023-02-05 12:22:00,2023-02-05 12:52:00,Fysisk fremmøde,Planlagt,2 Ambulant,DZ001,,...,2023-02-05 12:22:00,178.703.779,OBST EFTERFØDSEL 30,Mand,Ugift,J,Danmark,Hovedstaden (84),2100,København


In [29]:

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4658066 entries, 0 to 4658065
Data columns (total 24 columns):
 #   Column                                   Dtype         
---  ------                                   -----         
 0   Patient ID                               category      
 1   Patientkontakt ID                        int64         
 2   Patient alder på kontaktstart tidspunkt  Int16         
 3   Kontakt startdato                        datetime64[ns]
 4   Kontakt slutdato                         datetime64[ns]
 5   Kontakttype                              category      
 6   Indlæggelsesmåde                         category      
 7   Patientkontakttype                       category      
 8   Aktionsdiagnosekode                      category      
 9   Bidiagnosekode                           category      
 10  Behandlingsansvarlig Afdeling            category      
 11  Kontakt varighed (timer)                 float32       
 12  Procedure-kode              

In [3]:
# Define a function to categorize time of day
def categorize_time_of_day(hour):
    if 5 <= hour < 12:
        return 'Morning'
    elif 12 <= hour < 17:
        return 'Afternoon'
    elif 17 <= hour < 21:
        return 'Evening'
    else:
        return 'Night'

# Extract the hour from 'Kontakt startdato' and categorize time of day
df['time_of_day'] = df['Kontakt startdato'].dt.hour.apply(categorize_time_of_day)

# Convert 'time_of_day' to numeric category with 1, 2, 3, 4
df['time_of_day'] = df['time_of_day'].astype('category').cat.codes

In [4]:
df['Patientkontakttype'].unique()

['2 Ambulant', '4 Ambulant Us/Op', '3 Akut Ambulant']
Categories (3, object): ['2 Ambulant', '3 Akut Ambulant', '4 Ambulant Us/Op']

In [ ]:

# Update the feature set X to include the one-hot encoded columns
X = df[['Patient postnummer', 'Patient køn', 'Patient alder på kontaktstart tidspunkt','time_of_day']]
X['Patient køn'] = X['Patient køn'].apply(lambda x: 1 if x == 'Mand' else 0)

# Define bins and labels for the target variable
bins = np.arange(0, 10.25, 0.25)  # Create bins from 0 to 10 hours with 15-minute intervals
labels = range(len(bins) - 1)  # Create labels for the bins

# Bin the target variable
df['binned_y'] = pd.cut(df['Kontakt varighed (timer)'], bins=bins, labels=labels, right=False)

# Drop rows with NaN values in the binned target variable
df = df.dropna(subset=['binned_y'])

# Update X and y after dropping NaN rows
X = X.loc[df.index]
y = df['binned_y']

# Fit the RandomForestClassifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X, y)


C:\Users\jacob\AppData\Local\Temp\ipykernel_6496\3416263290.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['Patient køn'] = X['Patient køn'].apply(lambda x: 1 if x == 'Mand' else 0)


In [44]:
clf.score(X, y)
# Output the feature importances
importances = clf.feature_importances_
feature_importances = pd.DataFrame(importances, index=X.columns, columns=['importance']).sort_values('importance', ascending=False)
print(feature_importances)

KeyboardInterrupt: 

In [47]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Prepare the feature set X


# Add the one-hot encoded time of day columns
def categorize_time_of_day(hour):
    if 5 <= hour < 12:
        return 'Morning'
    elif 12 <= hour < 17:
        return 'Afternoon'
    elif 17 <= hour < 21:
        return 'Evening'
    else:
        return 'Night'

df['time_of_day'] = df['Kontakt startdato'].dt.hour.apply(categorize_time_of_day)
# Convert 'time_of_day' to numeric category with 1, 2, 3, 4
df['time_of_day'] = df['time_of_day'].astype('category').cat.codes

X = df[['Patient postnummer', 'Patient køn', 'Patient alder på kontaktstart tidspunkt', 'time_of_day']]
X['Patient køn'] = X['Patient køn'].apply(lambda x: 1 if x == 'Mand' else 0)

# Define the target variable y
y = df['Kontakt varighed (timer)']

# Drop rows with NaN values in X or y
X = X.dropna()
y = y.loc[X.index]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Fit the RandomForestRegressor
regressor = RandomForestRegressor(n_estimators=100, random_state=42)
regressor.fit(X_train, y_train)

# Evaluate the model
y_pred = regressor.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

# Feature importance
importances = regressor.feature_importances_
feature_importances = pd.DataFrame(importances, index=X.columns, columns=['importance']).sort_values('importance', ascending=False)
print(feature_importances)

C:\Users\jacob\AppData\Local\Temp\ipykernel_12280\4191083637.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['time_of_day'] = df['Kontakt startdato'].dt.hour.apply(categorize_time_of_day)
C:\Users\jacob\AppData\Local\Temp\ipykernel_12280\4191083637.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['time_of_day'] = df['time_of_day'].astype('category').cat.codes
C:\Users\jacob\AppData\Local\Temp\ipykernel_12280\4191083637.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a

Mean Squared Error: 1.8390562399792971
                                         importance
Patient postnummer                         0.422424
Patient alder på kontaktstart tidspunkt    0.365566
time_of_day                                0.167839
Patient køn                                0.044171
